<a href="https://colab.research.google.com/github/mayutakizawa-78/bert-MLM/blob/main/BERT_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BERT を用いた自動文章生成

自然気語処理のBERTモデル（　https://arxiv.org/abs/1810.04805　）を用いた文章の穴埋めを自動予測するコードを生成。

### 手順
### 1. ライブラリのインストール
### 2. モデルの準備
### 3. マスク単語予測(MLM)

今回は、例文【アルベルト・アインシュタインはそれまでの物理学の認識を根本から変え、「20世紀最高の物理学者」とも評される】を用いて検証する。
入力テキストを変更して自由に文章を設定可能。
- input_ids の f' 'の中に書き込む
- {tokenizer.mask_token}を含めるのを忘れずに

### 1. ライブラリのインストール

In [1]:
# 日本語用パッケージをインストール
!pip install "fugashi[unidic]"
!pip install ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.9/615.9 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7425 sha256=93ce23231ac24d498fb8802b3f82f47bd41b3b88062ab3b32c5f62c2d5a5b944
  Stored in directory: /root/.cache/pip/wheels/22/bc/bb/46aba36d0388f67dfe44bb0edc20a2c964560d4d19ec394e05
Successfully built unidic
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 47.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=5f541d7d7bbf6e62448e707144a0faaed61260060205fa8f57d0fed97eb486f8
  Stored in directory: /root/.cache/pip/wheels/45/b7/f5/a21e68db846eedcd00d69e37d60bab3f68eb20b1d99cdff652
Successfully bui

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.3 MB/s eta 0:00:00


In [3]:
pip install fugashi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 2. モデルの準備

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

# トークナイザの準備
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
# モデルの準備
model = AutoModelForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 3. マスク単語予測(MLM)

In [10]:
# 入力テキストのエンコード
input_ids = tokenizer.encode(f'アルベルト・アインシュタインはそれまでの物理学の認識を根本から変え、「20世紀最高の{tokenizer.mask_token}」とも評される。', return_tensors='pt')
print('input_ids:', tokenizer.convert_ids_to_tokens(input_ids[0].tolist()))

# マスクインデックスの取得
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
print('masked_index:', masked_index)

# マスクトークンの予測
result = model(input_ids)

pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]

# 5回繰り返す
for pred_id in pred_ids:
    output_ids = input_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    print(tokenizer.decode(output_ids))

input_ids: ['[CLS]', 'アルベルト', '・', 'アインシュタイン', 'は', 'それ', 'まで', 'の', '物理', '学', 'の', '認識', 'を', '根本', 'から', '変え', '、', '「', '20', '世紀', '最高', 'の', '[MASK]', '」', 'と', 'も', '評さ', 'れる', '。', '[SEP]']
masked_index: 22
[CLS] アルベルト ・ アインシュタイン は それ まで の 物理 学 の 認識 を 根本 から 変え 、 「 20 世紀 最高 の アインシュタイン 」 と も 評さ れる 。 [SEP]
[CLS] アルベルト ・ アインシュタイン は それ まで の 物理 学 の 認識 を 根本 から 変え 、 「 20 世紀 最高 の 天才 」 と も 評さ れる 。 [SEP]
[CLS] アルベルト ・ アインシュタイン は それ まで の 物理 学 の 認識 を 根本 から 変え 、 「 20 世紀 最高 の 学者 」 と も 評さ れる 。 [SEP]
[CLS] アルベルト ・ アインシュタイン は それ まで の 物理 学 の 認識 を 根本 から 変え 、 「 20 世紀 最高 の 科学 」 と も 評さ れる 。 [SEP]
[CLS] アルベルト ・ アインシュタイン は それ まで の 物理 学 の 認識 を 根本 から 変え 、 「 20 世紀 最高 の 哲学 」 と も 評さ れる 。 [SEP]
